In [35]:
from matplotlib import pyplot as plt
from pprint import pprint

import json
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict, namedtuple
from p4utils.utils.helper import load_topo
import numpy as np
from collections import Counter


if not os.path.abspath('.').endswith('netscope'):
    root_path = os.path.dirname(os.path.dirname(os.path.abspath('.')))
    os.chdir(root_path)
    from analysis.load import Loader

from evaluation.spidermon.SpiderMon import load2df
import itertools

In [36]:
def rater(exp_kind, answer, tdf, topo):
    ans = answer[exp_kind][0]
    flow_gt = f"{ans['src']}-{ans['dst']}"
    gt = ans['groundTruth']
    
    index = 0
    if exp_kind == "burst":
        flow_list = tdf.sort_values(['deq_timedelta'], ascending=False).drop_duplicates(['flow'], keep="first").flow.tolist()
        if flow_gt in flow_list:
            index = flow_list.index(flow_gt) + 1
    elif exp_kind == "ecmp_imbalance":
        f_df = tdf.sort_values(['deq_timedelta'], ascending=False).drop_duplicates(['flow'], keep="first").reset_index(drop=True)
        for idx, flow in f_df.iterrows():
            src, dst = flow.flow.split('-')
            paths = topo.get_shortest_paths_between_nodes(src, dst)
            sws = list(set(itertools.chain(*paths)))
            if gt in sws:
                index = idx + 1
                break
        # swdf = tdf.sort_values(['deq_timedelta'], ascending=False).drop_duplicates(['sw_id', 'ig_port'], keep="first").reset_index(drop=True)
        # swdf['prev'] = swdf.apply(lambda r: topo.port_to_node(f"s{r.sw_id}", r.ig_port), axis=1)
        # fork_sw = ans['groundTruth']
        # fork_df = swdf[swdf.prev==fork_sw]
        # for idx, row in fork_df.iterrows():
        #     # print(row)
        #     for p in ans['paths']:
        #         sws = p.strip(',').split(',')
        #         fork_id = sws.index(fork_sw)
        #         if sws[fork_id+1] == f"s{row.sw_id}":
        #             index = idx + 1
        #             break
        #     if index != 0:
        #         break
    elif exp_kind == "port_queue":
        port_list = tdf.sort_values(['deq_timedelta'], ascending=False).drop_duplicates(['port'], keep="first").port.tolist()
        if gt in port_list:
            index = port_list.index(gt) + 1
    elif exp_kind == "delay":
        port_list = tdf.sort_values(['next_link_delay'], ascending=False).drop_duplicates(['port'], keep="first").port.tolist()
        if gt in port_list:
            index = port_list.index(gt) + 1
    return index

In [39]:
# log_folder = "/home/user/dds/netscope/experiment/data/slot2/s2l4/burst/"
log_folder = "/mnt/netscope/data/slot2/s2l4/ecmp_imbalance/"
# log_folder = "/mnt/netscope/data/slot2/s2l4/burst2/"
# log_folder = "/mnt/netscope/data/slot2/fattree4/burst/"
# log_folder = "/mnt/netscope/data/slot2/fattree4/port_queue3/"
# log_folder = "/home/user/dds/netscope-data/slot2/s2l4/ecmp_imbalance"
# log_folder = "/mnt/netscope/data/slot2/fattree4/delay/"

rca_result = []
exp_ids = sorted(os.listdir(log_folder))#[:7]
for ie, exp_id in enumerate(exp_ids):
    print(ie, exp_id, end="...")
    log_dir = os.path.join(log_folder, exp_id)
    
    with open(os.path.join(log_dir, 'answer.json'), 'r') as f:
        answer = json.load(f)
    exp_kind = list(answer.keys())[0]
    
    
    loader = Loader(log_dir)
    topo = loader.get_topo()
    hosts = loader.load_hosts(debug=True)
    
    sw_dfs = load2df(hosts, topo)
    tdf = sw_dfs.copy(deep=True)
    index = rater(exp_kind, answer, tdf, topo)
    print(index)
    rca_result.append(dict(
        id=exp_id,
        index=index,
        # flow_list=flow_list,
        answer=answer,    
    ))
    # break

0 20221005_0935GMT...1
1 20221005_0951GMT...1
2 20221005_0955GMT...2
3 20221005_1000GMT...3
4 20221005_1042GMT...1
5 20221005_1047GMT...1
6 20221005_1052GMT...4
7 20221005_1054GMT...3
8 20221005_1109GMT...1
9 20221005_1131GMT...1
10 20221005_1133GMT...2
11 20221005_1146GMT...1
12 20221005_1148GMT...2
13 20221005_1153GMT...2
14 20221005_1156GMT...3
15 20221005_1200GMT...1
16 20221005_1203GMT...1
17 20221005_1210GMT...1
18 20221005_1213GMT...2
19 20221005_1225GMT...1
20 20221005_1227GMT...3
21 20221005_1237GMT...2
22 20221005_1239GMT...2
23 20221005_1244GMT...1


In [40]:
answer

{'ecmp_imbalance': [{'src': 'h1',
   'dst': 'h3',
   'groundTruth': 's3',
   'paths': ['s3,s1,s4,', 's3,s2,s4,'],
   'abnormalKind': 'ecmp imbalance'}]}

In [41]:
def topk(k, counter):
    return sum([counter.get(i, 0) for i in range(1, k+1)])
rca_counts = Counter([r['index'] for r in rca_result])
print(rca_counts)
L = len(rca_result)

for k in [1, 2, 3, 5, 10]:
    tk = topk(k, rca_counts)
    print(f"top{k}: {tk}/{L} ({tk/L*100:.1f}%)")
    if tk == L: break

Counter({1: 12, 2: 7, 3: 4, 4: 1})
top1: 12/24 (50.0%)
top2: 19/24 (79.2%)
top3: 23/24 (95.8%)
top5: 24/24 (100.0%)


In [6]:
pd_code = 'dict(name="SyNDB", '
for k in [1, 2, 3, 5, 10]:
    tk = topk(k, rca_counts)
    pd_code += f"t{k}={tk}/L, "
    if tk == L: break
pd_code = pd_code.rstrip(", ")+"),"
print(pd_code)

dict(name="SyNDB", t1=31/L),


In [7]:
exam_score = 0
for index, count in rca_counts.items():
    exam_score += (index - 1) * count
print(exam_score, exam_score/L)

0 0.0


In [8]:

sw_dfs.sort_values(['deq_timedelta'], ascending=False)

,sw_id,ig_port,eg_port,state_i,state,timestamp,qdepth,packet_length,enq_timestamp,deq_timedelta,enq_qdepth,flow,flow_t,pid,next_link_delay,port
10874,3,4,5,6,7,258035519076962,63,393,2473868779,15986609,63,h9-h13,258035519071317,346,2022.0,"s3,s18,"
11905,3,2,5,6,7,258035525589211,63,336,2480381603,15973613,63,h3-h15,258035525580682,193,2641.0,"s3,s18,"
11627,3,2,5,6,7,258035523857733,63,336,2478650587,15954678,63,h3-h15,258035523847573,189,4637.0,"s3,s18,"
11747,3,2,5,6,7,258035524624546,63,364,2479416999,15938228,63,h1-h16,258035524616066,189,3290.0,"s3,s18,"
10777,3,2,5,6,7,258035518376809,63,336,2473169825,15935855,63,h3-h15,258035518370817,176,4598.0,"s3,s18,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10398,16,2,4,6,7,258035516113881,0,430,2470905762,21,0,h4-h11,258035516092812,284,NaN,"s16,h11,"
1924,5,3,5,6,7,258035464319083,0,365,2419111582,21,0,h5-h3,258035464306722,29,2584.0,"s5,s8,"
9374,10,2,4,6,7,258035509904014,0,346,2464695828,21,0,h14-h5,258035509884715,119,2101.0,"s10,s11,"
2816,2,2,5,6,7,258035469783596,0,364,2424575368,20,0,h1-h16,258035469771876,46,1841.0,"s2,s17,"
